<h1><center>Explore the Rainforest Soundscape</center></h1> 


<center><img src="https://upload.wikimedia.org/wikipedia/commons/a/a8/CRTO_Matthew-Gable.JPG" width="400"></img></center>

<h1 style='background:#E3C6AD; border:0; color:black'><center>Introduction</center></h1> 


In this Notebook, I will:
* Explore the **RFCX Species Audio Detection** dataset;
* Review various techniques for signal augmentation in parallel with exploring the sounds of the rainforest.  

<a id="0"></a>
### Content
* <a href='#1'>Analysis preparation</a>  
* <a href='#2'>Data ingestion</a>    
* <a href='#3'>Data exploration</a>    
* <a href='#4'>Sounds of the rainforest</a>    
* <a href='#5'>References</a>    

In the final section of the Notebook, you can find additional resources for data exploration, signal data augmentation as well as few baseline models. 

<a id="1"></a><h1 style='background:#135E46; border:0; color:black'><center>Analysis preparation</center></h1>

In [ ]:
import numpy as np
import pandas as pd
import librosa
import librosa.display
import soundfile as sf
import scipy.signal as signal
import matplotlib.pyplot as plt
import seaborn as sns
import IPython.display as ipd
from IPython.display import YouTubeVideo

<a href="#0"><small>Go to top</small></a>

<a id="2"></a><h1 style='background:#478966; border:0; color:black'><center>Data ingestion</center></h1> 

In [ ]:
data_tp_df=pd.read_csv('/kaggle/input/rfcx-species-audio-detection/train_tp.csv')
data_fp_df=pd.read_csv('/kaggle/input/rfcx-species-audio-detection/train_fp.csv')

In [ ]:
data_tp_df.head()

In [ ]:
data_fp_df.head()

<a href="#0"><small>Go to top</small></a>

<a id="3"></a><h1 style='background:#73A788; border:0; color:black'><center>Data exploration</center></h1> 

We start by glimpsing the data (press `Output` buton to review the TP and FP train sets).

In [ ]:
data_tp_df.info()

In [ ]:
data_fp_df.info()

In [ ]:
def plot_count(feature, title, df, size=1):
    '''
    Plot count of classes / feature
    param: feature - the feature to analyze
    param: title - title to add to the graph
    param: df - dataframe from which we plot feature's classes distribution 
    param: size - default 1.
    '''
    f, ax = plt.subplots(1,1, figsize=(4*size,4))
    total = float(len(df))
    g = sns.countplot(df[feature], order = df[feature].value_counts().index[:30], palette='Set1')
    g.set_title("Number and percentage of {}".format(title))
    if(size > 2):
        plt.xticks(rotation=90, size=8)
    for p in ax.patches:
        height = p.get_height()
        ax.text(p.get_x()+p.get_width()/2.,
                height + 3,
                '{:1.2f}%'.format(100*height/total),
                ha="center") 
    plt.show()  

In [ ]:
plot_count('species_id', 'TP: Species ID', data_tp_df, size=4)

In [ ]:
plot_count('songtype_id', 'TP: Songtype ID', data_tp_df, size=2)

In [ ]:
plot_count('species_id', 'FP: Species ID', data_fp_df, size=4)

In [ ]:
plot_count('songtype_id', 'FP: Songtype ID', data_fp_df, size=2)

In [ ]:
def plot_feature_distribution(data_df, feature, feature2, title, kde_mode=False, hist_mode=True):
    f, ax = plt.subplots(1,1, figsize=(12,6))
    for item in list(data_df[feature2].unique()):
        d_df = data_df.loc[data_df[feature2]==item]
        try:
            sns.distplot(d_df[feature], kde=kde_mode, hist=hist_mode, label=item)
        except:
            pass
    plt.legend(labels=list(data_df[feature2].unique()), bbox_to_anchor=(1, 1), loc='upper right', ncol=2)
    plt.title(title)
    plt.show()

In [ ]:
plot_feature_distribution(data_tp_df, 'f_min', 'species_id', "Minimum frequency distribution, TP data, grouped by species id")

In [ ]:
plot_feature_distribution(data_tp_df, 'f_max', 'species_id', "Maximum frequency distribution, TP data, grouped by species id")

In [ ]:
plot_feature_distribution(data_fp_df, 'f_min', 'species_id', "Minimum frequency distribution, FP data, grouped by species id")

In [ ]:
plot_feature_distribution(data_fp_df, 'f_max', 'species_id', "Maximum frequency distribution, FP data, grouped by species id")

In [ ]:
plot_feature_distribution(data_tp_df, 'f_min', 'songtype_id', "Minimum frequency distribution, TP data, grouped by songtype id")

In [ ]:
plot_feature_distribution(data_tp_df, 'f_max', 'songtype_id', "Maximum frequency distribution, TP data, grouped by songtype id")

In [ ]:
plot_feature_distribution(data_fp_df, 'f_min', 'songtype_id', "Minimum frequency distribution, FP data, grouped by songtype id")

In [ ]:
plot_feature_distribution(data_fp_df, 'f_max', 'songtype_id', "Maximum frequency distribution, FP data, grouped by songtype id")

In [ ]:
plot_feature_distribution(data_tp_df, 't_min', 'species_id', 
                          "Minimum time distribution, TP data, grouped by species id", kde_mode=True, hist_mode=False)

In [ ]:
plot_feature_distribution(data_tp_df, 't_max', 'species_id', 
                          "Max time distribution, TP data, grouped by species id", kde_mode=True, hist_mode=False)

In [ ]:
plot_feature_distribution(data_tp_df, 't_min', 'songtype_id', 
                          "Minimum time distribution, TP data, grouped by songtype id", kde_mode=True, hist_mode=False)

In [ ]:
plot_feature_distribution(data_tp_df, 't_max', 'songtype_id', 
                          "Max time distribution, TP data, grouped by songtype id", kde_mode=True, hist_mode=False)

In [ ]:
plot_feature_distribution(data_fp_df, 't_min', 'species_id', 
                          "Minimum time distribution, FP data, grouped by species id", kde_mode=True, hist_mode=False)

In [ ]:
plot_feature_distribution(data_fp_df, 't_max', 'species_id', 
                          "Max time distribution, FP data, grouped by species id", kde_mode=True, hist_mode=False)

In [ ]:
plot_feature_distribution(data_fp_df, 't_min', 'songtype_id', 
                          "Minimum time distribution, FP data, grouped by songtype id", kde_mode=True, hist_mode=False)

In [ ]:
plot_feature_distribution(data_fp_df, 't_max', 'songtype_id', 
                          "Max time distribution, FP data, grouped by songtype id", kde_mode=True, hist_mode=False)

<a href="#0"><small>Go to top</small></a>

<a id="4"></a><h1 style='background:#E3C6AD; border:0; color:black'><center>Sounds of the rainforest</center></h1> 

In [ ]:
YouTubeVideo("7iUrEOy3coA", width=800, height=450)

We define functions for display of:
- Waveplots
- Spectrograms  
- Mel spectrograms  
- Chroma feature
- Harmonics and Perceptual sound wave components

<div class="alert alert-block alert-info">
<b>Note:</b> We can build features for our model using the functions defined here.
</div>
<br>
We also define a function for playing the sound.

We then show Waveplots, Spectrograms, Mel Spectrograms, Chroma feature and combined Harmonics and Perceptual graphs for few of the recordings, from both the TP and FP train sets.

In [ ]:
def plot_audio_file(data_df, idx):
    audio_file_path = '/kaggle/input/rfcx-species-audio-detection/train/'+data_df.recording_id[idx]+'.flac'
    plt.figure(figsize=(12,6))
    x , sr = librosa.load(audio_file_path)
    librosa.display.waveplot(x, sr=sr)
    plt.gca().set_title(f"Waveplot - file: {data_df.recording_id[idx]}")
    plt.show()

In [ ]:
def plot_spectrogram(data_df, idx):
    audio_file_path = '/kaggle/input/rfcx-species-audio-detection/train/'+data_df.recording_id[idx]+'.flac'
    plt.figure(figsize=(12,6))
    x , sr = librosa.load(audio_file_path)
    xs = librosa.stft(x)
    xdb = librosa.amplitude_to_db(abs(xs))
    librosa.display.specshow(xdb, sr=sr, x_axis='time', y_axis='hz')
    plt.gca().set_title(f"Spectrogram - file: {data_df.recording_id[idx]}")
    plt.colorbar()

In [ ]:
def plot_mel_spectrogram(data_df, idx):
    audio_file_path = '/kaggle/input/rfcx-species-audio-detection/train/'+data_df.recording_id[idx]+'.flac'
    plt.figure(figsize=(12,6))
    x , sr = librosa.load(audio_file_path)
    xs = librosa.feature.melspectrogram(x)
    xdb = librosa.amplitude_to_db(abs(xs))
    librosa.display.specshow(xdb, sr=sr, x_axis='time', y_axis='hz')
    plt.gca().set_title(f"Mel spectrogram - file: {data_df.recording_id[idx]}")
    plt.colorbar()

In [ ]:
def plot_harmonics_and_perceptual(data_df, idx):
    audio_file_path = '/kaggle/input/rfcx-species-audio-detection/train/'+data_df.recording_id[idx]+'.flac'
    plt.figure(figsize=(12,6))
    x , sr = librosa.load(audio_file_path)
    y_harmonics, y_perceptual = librosa.effects.hpss(x)
    plt.plot(y_perceptual, color = '#BBAA12')
    plt.plot(y_harmonics, color = '#12AABB')
    plt.legend(("Perceptual", "Harmonics"))
    plt.title(f"Harmonics and Perceptual - file: {data_df.recording_id[idx]}")

In [ ]:
def plot_chroma_feature(data_df, idx):
    hop_length=12
    audio_file_path = '/kaggle/input/rfcx-species-audio-detection/train/'+data_df.recording_id[idx]+'.flac'
    plt.figure(figsize=(12,6))
    x , sr = librosa.load(audio_file_path)
    chromagram = librosa.feature.chroma_stft(x)
    librosa.display.specshow(chromagram, sr=sr, x_axis='time', y_axis='chroma',hop_length=hop_length, cmap='coolwarm')
    plt.title(f"Chroma feature - file: {data_df.recording_id[idx]}")

In [ ]:
def play_sound(data_df, idx):
    audio_file_path = '/kaggle/input/rfcx-species-audio-detection/train/'+data_df.recording_id[idx]+'.flac'
    return ipd.Audio(audio_file_path)

<a href="#0"><small>Go to top</small></a>

<h2 style='background:#D09D7B; border:0; color:black'><center>Sound samples from TP set</center></h2> 

In [ ]:
plot_audio_file(data_tp_df, 20)

In [ ]:
plot_spectrogram(data_tp_df, 20)

In [ ]:
plot_mel_spectrogram(data_tp_df, 20)

In [ ]:
plot_chroma_feature(data_tp_df, 20)

In [ ]:
plot_harmonics_and_perceptual(data_tp_df, 20)

In [ ]:
play_sound(data_tp_df, 20)

In [ ]:
plot_audio_file(data_tp_df, 50)

In [ ]:
plot_spectrogram(data_tp_df, 50)

In [ ]:
plot_mel_spectrogram(data_tp_df, 50)

In [ ]:
plot_chroma_feature(data_tp_df, 50)

In [ ]:
plot_harmonics_and_perceptual(data_tp_df, 50)

In [ ]:
play_sound(data_tp_df, 50)

In [ ]:
plot_audio_file(data_tp_df, 100)

In [ ]:
plot_spectrogram(data_tp_df, 100)

In [ ]:
plot_mel_spectrogram(data_tp_df, 100)

In [ ]:
plot_chroma_feature(data_tp_df, 100)

In [ ]:
plot_harmonics_and_perceptual(data_tp_df, 100)

In [ ]:
play_sound(data_tp_df, 100)

<a href="#0"><small>Go to top</small></a>

<h2 style='background:#B67B65; border:0; color:black'><center>Sound samples from FP set</center></h2> 

In [ ]:
plot_audio_file(data_fp_df, 10)

In [ ]:
plot_spectrogram(data_fp_df, 10)

In [ ]:
plot_mel_spectrogram(data_fp_df, 10)

In [ ]:
plot_chroma_feature(data_fp_df, 10)

In [ ]:
plot_harmonics_and_perceptual(data_fp_df, 10)

In [ ]:
play_sound(data_fp_df, 10)

In [ ]:
plot_audio_file(data_fp_df, 70)

In [ ]:
plot_spectrogram(data_fp_df, 70)

In [ ]:
plot_mel_spectrogram(data_fp_df, 70)

In [ ]:
plot_chroma_feature(data_fp_df, 70)

In [ ]:
plot_harmonics_and_perceptual(data_fp_df, 70)

In [ ]:
play_sound(data_fp_df, 70)

In [ ]:
plot_audio_file(data_fp_df, 100)

In [ ]:
plot_spectrogram(data_fp_df, 100)

In [ ]:
plot_mel_spectrogram(data_fp_df, 100)

In [ ]:
plot_chroma_feature(data_fp_df, 100)

In [ ]:
plot_harmonics_and_perceptual(data_fp_df, 100)

In [ ]:
play_sound(data_fp_df, 100)

<img src="https://upload.wikimedia.org/wikipedia/commons/4/48/Hyperolius_tuberculatus.jpg" width="400"></img>

<a href="#0"><small>Go to top</small></a>

<a id="5"></a><h1 style='background:#135E46; border:0; color:black'><center>References</center></h1>   

I include here various useful resources: papers, reviews, projects, trivia, notebooks.

<h2 style='background:#478966; border:0; color:black'><center>Papers, reviews and projects</center></h2>   

* [A pipeline for identification of bird and frog species in tropical soundscape recordings using a convolutional neural network](https://www.sciencedirect.com/science/article/pii/S1574954120300637)

* [Frog Sound Identification System for Frog Species Recognition](https://link.springer.com/chapter/10.1007/978-3-642-36642-0_5)  

* [Bird recognition - review of useful resources](https://github.com/AgaMiko/bird-recognition-review)   

* [Detecting Frog Calling Activity Based on Acoustic Event Detection and Multi-label Learning](https://www.sciencedirect.com/science/article/pii/S1877050916307700)

* [Acoustic classification of Australian frogs for ecosystem
surveys](https://eprints.qut.edu.au/103530/1/Jie_Xie_Thesis.pdf)  

* [Data Augmentation Review](https://github.com/AgaMiko/data-augmentation-review)    

* [Automatic recognition of frog calls using a multi-stage average spectrum](https://www.sciencedirect.com/science/article/pii/S0898122112002763)  

* [A Baseline for Large-Scale Bird Species Identification in Field Recordings](http://ceur-ws.org/Vol-2125/paper_85.pdf).  

* [Identifying Birds by Sound: Large-scale Acoustic Event Recognition for Avian Activity Monitoring](https://monarch.qucosa.de/api/qucosa%3A36986/attachment/ATT-0/)   

* [LifeCLEF 2019: Biodiversity Identification and Prediction Challenges](https://www.researchgate.net/profile/Henning_Mueller2/publication/331733244_LifeCLEF_2019_Biodiversity_Identification_and_Prediction_Challenges/links/5c8a35e592851c1df9407d46/LifeCLEF-2019-Biodiversity-Identification-and-Prediction-Challenges.pdf)  

<h2 style='background:#478966; border:0; color:black'><center>More resources</center></h2>   

* [Calls of Frogs and Toads of the Northeast](https://musicofnature.com/calls-of-frogs-and-toads-of-the-northeast/)  

* [Sounds of Frogs](https://www.fws.gov/refuge/clarks_river/sounds_of_frogs.html)   

* [Singing in the Rain Forest: How a Tropical Bird Song Transfers Information](https://journals.plos.org/plosone/article?id=10.1371/journal.pone.0001580)

<h2 style='background:#478966; border:0; color:black'><center>Trivia</center></h2>   


* [Shazam for Birds: Three Apps That Recognize Bird Calls](https://lifehacker.com/shazam-for-birds-three-apps-that-recognize-bird-calls-1797955537)  

* [The Shazam App For Frogs!](https://www.iheartradio.ca/virginradio/calgary/trending/the-shazam-app-for-frogs-1.3430016)   

* [Song Sleuth -  Helping you become a better birder](https://www.songsleuth.com/#/)  

* [BirdNET - Bird sound identification](https://play.google.com/store/apps/details?id=de.tu_chemnitz.mi.kahst.birdnet)


<h2 style='background:#478966; border:0; color:black'><center>Notebooks</center></h2>   


* [Audio Data Analysis Using librosa](https://www.kaggle.com/hamditarek/audio-data-analysis-using-librosa)  

* [Birdcall Recognition: EDA and Audio FE](https://www.kaggle.com/andradaolteanu/birdcall-recognition-eda-and-audio-fe)    

* [EDA and Audio Processing with Python](https://www.kaggle.com/parulpandey/eda-and-audio-processing-with-python)  

* [Rainforest Connection Analysis Using librosa](https://www.kaggle.com/hamditarek/rainforest-connection-analysis-using-librosa)   

* [RFCX: Audio Data Augmentation(Japanese+English)](https://www.kaggle.com/hidehisaarai1213/rfcx-audio-data-augmentation-japanese-english)  

* [Tabular XGboost GPU + FFT GPU + Cuml = FAST](https://www.kaggle.com/titericz/0-525-tabular-xgboost-gpu-fft-gpu-cuml-fast)   

<a href="#0"><small>Go to top</small></a>